In [64]:
import torchaudio
import torch 
import pandas as pd
import numpy as np
import re 
import os 
from torch.nn import functional as F
import random
import math
import pickle
import gc
import torch.nn as nn
from tqdm.notebook import tqdm
import os
import glob
import warnings
from torch.utils.data import Dataset, DataLoader
warnings.filterwarnings(action='ignore')

In [65]:
base_dir = '/home/work/'
np1 = glob.glob(base_dir+'mel_dataset1/*.npy')
np2 = glob.glob(base_dir+'mel_dataset2/*.npy')
np3 = glob.glob(base_dir+'mel_dataset3/*.npy')
np4 = glob.glob(base_dir+'mel_dataset4/*.npy')
np5 = glob.glob(base_dir+'mel_dataset5/*.npy')
np6 = glob.glob(base_dir+'mel_dataset6/*.npy')
np7 = glob.glob(base_dir+'mel_dataset7/*.npy')
np8 = glob.glob(base_dir+'mel_dataset8/*.npy')

In [66]:
total = np1+np2+np3+np4+np5+np6+np7+np8 

In [67]:
train_csv = pd.read_csv('new_train.csv')
valid_csv = pd.read_csv('new_valid.csv')
all_csv =pd.read_csv('new_all.csv')

In [68]:
def npy_list(csv):
    npy_list = []
    for song_path in tqdm(csv['npy_path']):
        song_npy = np.load(song_path).squeeze()
        npy_list.append(song_npy)
    return npy_list


train_list = npy_list(train_csv)
valid_list = npy_list(valid_csv)


  0%|          | 0/3825 [00:00<?, ?it/s]

  0%|          | 0/675 [00:00<?, ?it/s]

In [69]:
all_list = train_list + valid_list

In [72]:
epochs = 150
batch_size = 32

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print(DEVICE)


cuda


In [73]:

# epoch문 안에 들어가야되나?
#train_batch_li = DataLoader(train_list, batch_size=batch_size, shuffle=True,drop_last=True)
#val_batch_li = DataLoader(valid_list, batch_size=batch_size, shuffle=True,drop_last=True)


In [78]:
def train(model, train_loader):
    model.train()
    train_loss = 0
    for batch in train_loader: # 배치파일 다 만들면 train_loader [] 삭제하기
        
        #mel = np.load(batch_name)
        mel = torch.FloatTensor(batch).to(DEVICE)
        
        optimizer.zero_grad()
        
        encode, output = model(mel)
        
        loss = criterion(output, mel)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        
    train_loss /= (len(train_loader))

    return train_loss

def val(model, train_loader):
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for batch in train_loader:# 배치파일 다 만들면 train_loader [] 삭제하기
            
            #mel = np.load(batch_name)
            mel = torch.FloatTensor(batch).to(DEVICE)

            encode, output = model(mel)

            loss = criterion(output, mel)

            val_loss += loss.item()

    val_loss /= (len(train_loader))

    return val_loss


def get_mel_embeding(model, train_loader):
    model.eval()
    mel_embeding_li = []
    with torch.no_grad():
        for batch in train_loader: # 배치파일 다 만들면 train_loader [] 삭제하기
            
            #mel = np.load(batch_name)
            mel = torch.FloatTensor(batch).to(DEVICE)
            
            encode, output = model(mel)
            mel_embeding_li.append(encode.detach().cpu().numpy())

    return mel_embeding_li

In [79]:
# encoder <- 과거의 데이터를 바탕으로 미래를 예측 (t-1의 인과성 학습)
# decoder <- 미래의 데이터를 바탕으로 과거를 예측 (t+1의 인과성 학습)
class TimeAutoEncoder(nn.Module):
    def __init__(self):
        super(TimeAutoEncoder, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels = 48, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 1),
            nn.BatchNorm1d(512),
            nn.ReLU(),
        )

        self.conv2 = nn.Sequential(
            nn.Conv1d(in_channels = 512, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 2),
            nn.BatchNorm1d(256),
            nn.ReLU(),
        )

        self.conv3 = nn.Sequential(
            nn.Conv1d(in_channels = 256, out_channels = 128, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            nn.BatchNorm1d(128),
            nn.ReLU(),
        )

        self.conv4 = nn.Sequential(
            nn.Conv1d(in_channels = 128, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        self.conv5 = nn.Sequential(
            nn.Conv1d(in_channels = 64, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        self.conv6 = nn.Sequential(
            nn.Conv1d(in_channels = 32, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            nn.BatchNorm1d(16),
            nn.ReLU(),
        )

        self.conv7 = nn.Sequential(
            nn.Conv1d(in_channels = 16, out_channels = 8, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            nn.BatchNorm1d(8),
            nn.ReLU(),
        )

        self.encoder_fc = nn.Sequential(
            nn.Linear(8 * 7501, 128),
            nn.BatchNorm1d(128),
            nn.Tanh(),
        )
        
        self.decoder_fc = nn.Sequential(
            nn.Linear(128, 8 * 7501),
            nn.ReLU(),
        )

        self.t_conv1 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 8, out_channels = 16, kernel_size  = 3, stride = 1, dilation=62),
            nn.Conv1d(in_channels = 8, out_channels = 16, kernel_size = 3, stride = 1, padding = 0, dilation = 64),
            nn.BatchNorm1d(16),
            nn.ReLU(),
        )

        self.t_conv2 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 16, out_channels = 32, kernel_size  = 3, stride = 1, dilation = 30),
            nn.Conv1d(in_channels = 16, out_channels = 32, kernel_size = 3, stride = 1, padding = 0, dilation = 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        self.t_conv3 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 32, out_channels = 64, kernel_size  = 3, stride = 1, dilation=14),
            nn.Conv1d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 0, dilation = 16),
            nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        self.t_conv4 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 64, out_channels = 128, kernel_size  = 3, stride = 1, dilation = 6),
            nn.Conv1d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 0, dilation = 8),
            nn.BatchNorm1d(128),
            nn.ReLU(),
        )

        self.t_conv5 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 128, out_channels = 256, kernel_size  = 3, stride = 1, dilation=2),
            nn.Conv1d(in_channels = 128, out_channels = 256, kernel_size = 3, stride = 1, padding = 0, dilation = 4),
            nn.BatchNorm1d(256),
            nn.ReLU(),
        )

        self.t_conv6 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 256, out_channels = 512, kernel_size  = 3, stride = 1, dilation = 1),
            nn.Conv1d(in_channels = 256, out_channels = 512, kernel_size = 3, stride = 1, padding = 0, dilation = 2),
            nn.BatchNorm1d(512),
            nn.ReLU(),
        )

        self.t_conv7 = nn.Sequential(
            # nn.ConvTranspose1d(in_channels = 512, out_channels = 48, kernel_size  = 3, stride = 1, dilation= 1),
            nn.Conv1d(in_channels = 512, out_channels = 48, kernel_size = 3, stride = 1, padding = 0, dilation = 1)
        )

    def forward(self, mel_spec):
        x = F.pad(mel_spec, pad = (2, 0, 0, 0))
        x = self.conv1(x)
        #print(x.shape)
        x = F.pad(x, pad = (4, 0, 0, 0))
        x = self.conv2(x)
        #print(x.shape)
        x = F.pad(x, pad = (8, 0, 0, 0))
        x = self.conv3(x)
        #print(x.shape)
        x = F.pad(x, pad = (16, 0, 0, 0))
        x = self.conv4(x)
        #print(x.shape)
        x = F.pad(x, pad = (32, 0, 0, 0))
        x = self.conv5(x)
        #print(x.shape)
        x = F.pad(x, pad = (64, 0, 0, 0))
        x = self.conv6(x)
        #print(x.shape)
        x = F.pad(x, pad = (128, 0, 0, 0))
        x = self.conv7(x)
        #print(x.shape)
        encode = self.encoder_fc(x.view(-1, 8 * 7501))

        #encode = self.encoder_fc(x.view(-1, 8 * 1876))

        # print('decode')
        x = self.decoder_fc(encode)
        x = x.view(-1, 8, 7501)
        x = torch.swapaxes(torch.fliplr(torch.swapaxes(x, 1, 2)), 1, 2)
        x = F.pad(x, pad = (128, 0, 0, 0))
        x = self.t_conv1(x)
        #print(x.shape)
        x = F.pad(x, pad = (64, 0, 0, 0))
        x = self.t_conv2(x)
        #print(x.shape)
        x = F.pad(x, pad = (32, 0, 0, 0))
        x = self.t_conv3(x)
        #print(x.shape)
        x = F.pad(x, pad = (16, 0, 0, 0))
        x = self.t_conv4(x)
        #print(x.shape)
        x = F.pad(x, pad = (8, 0, 0, 0))
        x = self.t_conv5(x)
        #print(x.shape)
        x = F.pad(x, pad = (4, 0, 0, 0))
        x = self.t_conv6(x)
        #print(x.shape)
        x = F.pad(x, pad = (2, 0, 0, 0))
        x = self.t_conv7(x)
        #print(x.shape)
        x = torch.swapaxes(torch.fliplr(torch.swapaxes(x, 1, 2)), 1, 2)
        
        return encode, x

In [80]:
model = TimeAutoEncoder().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = nn.MSELoss()

In [85]:
import time

model_dir = '/home/work/Tcae_apply/model_dir/'
min_loss = 987654321

train_batch_li = DataLoader(train_list, batch_size=batch_size, shuffle=True,drop_last=True)
val_batch_li = DataLoader(valid_list, batch_size=batch_size, shuffle=True,drop_last=True)

for epoch in tqdm(range(1, epochs + 1)):
    start = time.time()
    
    #train_batch_li = DataLoader(train_list, batch_size=batch_size, shuffle=True,drop_last=True)
    #val_batch_li = DataLoader(valid_list, batch_size=batch_size, shuffle=True,drop_last=True)

    train_loss = train(model = model, train_loader = train_batch_li) 
    val_loss = val(model = model, train_loader = val_batch_li) 
    end = time.time()

    print(f'EPOCH:{epoch}, Train Loss:{train_loss}, Val Loss:{val_loss}, 학습 시간: {end - start}')
    if val_loss < min_loss:
        min_loss = val_loss
        torch.save(model.state_dict(), model_dir + f'TimeAutoEncoder_val.pt')
        print('모델 저장')
        
    #del train_batch_li
    #del valid_batch_li
 

  0%|          | 0/150 [00:00<?, ?it/s]

EPOCH:1, Train Loss:17833.780691964286, Val Loss:18560.346819196428, 학습 시간: 26.33114504814148
모델 저장
EPOCH:2, Train Loss:17316.818523503152, Val Loss:17546.024460565477, 학습 시간: 25.320499897003174
모델 저장
EPOCH:3, Train Loss:16773.10792246586, Val Loss:17579.230143229168, 학습 시간: 25.92996597290039
EPOCH:8, Train Loss:14877.210822610294, Val Loss:17489.569847470237, 학습 시간: 26.00956082344055
EPOCH:9, Train Loss:14505.108587184874, Val Loss:17390.005905877977, 학습 시간: 25.926118850708008
EPOCH:10, Train Loss:14236.222007943803, Val Loss:17526.696475074405, 학습 시간: 25.95997977256775
EPOCH:11, Train Loss:13988.963559447217, Val Loss:17471.2265625, 학습 시간: 25.54423451423645
EPOCH:12, Train Loss:13595.595013786764, Val Loss:17938.92354910714, 학습 시간: 25.473011016845703
EPOCH:13, Train Loss:13273.61115579044, Val Loss:17648.369047619046, 학습 시간: 25.53292155265808
EPOCH:14, Train Loss:13013.753791360294, Val Loss:17781.6875, 학습 시간: 25.98278832435608
EPOCH:15, Train Loss:12692.494415539653, Val Loss:17711.

KeyboardInterrupt: 

In [ ]:
#model = TimeAutoEncoder().to(DEVICE)
#model.load_state_dict(torch.load(model_dir + f'TimeAutoEncoder_val.pt', map_location = DEVICE))

In [ ]:
#mel_embeding_li = get_mel_embeding(model = model, train_loader = test_batch_li)
#mel_embeding = np.concatenate(mel_embeding_li, axis = 0)

In [ ]:
#np.save(batch_data_dir + 'mel_embeding_val.npy', mel_embeding)